In [3]:
# Import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [4]:
path1 = r"C:\Users\abebe\Documents\George Mason University\6- SUMME 2023\Team-SkySci-Analytics\Metadata Merge Transform\ground_weather_merged.csv"
path2 = r"C:\Users\abebe\Documents\George Mason University\6- SUMME 2023\Team-SkySci-Analytics\Atmospheric Data Scraper\atmospheric_data_master.csv"

In [16]:
atmosphericData = pd.read_csv(path2)
groundWeatehrData = pd.read_csv(path1)
atmosphericData.tail()
# pressure units seems like is units of pascal

,Timestamp,Latitude,Longitude,Temperature,Humidity,Pressure
7682683,2022-12-23 23:00:00,38.066033,-75.512216,211.694427,1.890851,30799.997711
7682684,2022-12-23 23:00:00,38.048988,-75.363237,211.131927,2.140851,30499.997711
7682685,2022-12-23 23:00:00,38.031813,-75.214305,210.569427,2.265851,30099.997711
7682686,2022-12-23 23:00:00,38.165567,-76.407075,213.444427,1.515851,36699.997711
7682687,2022-12-23 23:00:00,39.934715,-75.160969,212.381927,1.890851,30099.997711


In [6]:
# Group the DataFrame by longitude and latitude
grouped_data = atmosphericData.groupby(['Longitude', 'Latitude'])

# Iterate over the groups
for group, group_df in grouped_data:
# Print longitude, latitude, and the corresponding data for each group to look at all the available longitude and latitude
    print("Longitude:", group[0])
    print("Latitude:", group[1])
    print("Grouped data:")
    print(group_df)
    print()

Longitude: -79.98575527109776
Latitude: 35.40078346599076
Grouped data:
                   Timestamp   Latitude  Longitude  Temperature  Humidity  \
950      2022-09-01 00:00:00  35.400783 -79.985755   205.100800  7.036933   
1855     2022-09-01 00:00:00  35.400783 -79.985755   205.432175  6.669195   
3259     2022-09-01 01:00:00  35.400783 -79.985755   205.244629  6.918959   
4170     2022-09-01 01:00:00  35.400783 -79.985755   205.269501  6.805294   
6566     2022-09-01 02:00:00  35.400783 -79.985755   205.346100  6.683174   
...                      ...        ...        ...          ...       ...   
7676122  2022-12-23 21:00:00  35.400783 -79.985755   208.602615  2.921849   
7677112  2022-12-23 22:00:00  35.400783 -79.985755   208.974991  2.935882   
7678513  2022-12-23 22:00:00  35.400783 -79.985755   209.642914  2.684650   
7680825  2022-12-23 23:00:00  35.400783 -79.985755   209.565125  2.685125   
7681736  2022-12-23 23:00:00  35.400783 -79.985755   209.631927  2.640851   

   

[5472 rows x 6 columns]

Longitude: -79.85330689314155
Latitude: 37.42977899066032
Grouped data:
                   Timestamp   Latitude  Longitude  Temperature  Humidity  \
1082     2022-09-01 00:00:00  37.429779 -79.853307   207.725800  4.286933   
1731     2022-09-01 00:00:00  37.429779 -79.853307   207.807175  4.294195   
3133     2022-09-01 01:00:00  37.429779 -79.853307   207.744629  4.293959   
3753     2022-09-01 01:00:00  37.429779 -79.853307   207.769501  4.305294   
6697     2022-09-01 02:00:00  37.429779 -79.853307   207.908600  4.183174   
...                      ...        ...        ...          ...       ...   
7675992  2022-12-23 21:00:00  37.429779 -79.853307   213.790115  1.421849   
7677857  2022-12-23 22:00:00  37.429779 -79.853307   214.474991  1.310882   
7678930  2022-12-23 22:00:00  37.429779 -79.853307   214.455414  1.309650   
7680957  2022-12-23 23:00:00  37.429779 -79.853307   214.502625  1.310125   
7681606  2022-12-23 23:00:00  37.429779 -79.853307   214

[5472 rows x 6 columns]

Longitude: -79.6877277278549
Latitude: 37.53453350099264
Grouped data:
                   Timestamp   Latitude  Longitude  Temperature  Humidity  \
1223     2022-09-01 00:00:00  37.534534 -79.687728   207.913300  4.161933   
1581     2022-09-01 00:00:00  37.534534 -79.687728   207.994675  4.044195   
2983     2022-09-01 01:00:00  37.534534 -79.687728   207.869629  4.168959   
3722     2022-09-01 01:00:00  37.534534 -79.687728   208.019501  4.055294   
6838     2022-09-01 02:00:00  37.534534 -79.687728   208.158600  3.933174   
...                      ...        ...        ...          ...       ...   
7675852  2022-12-23 21:00:00  37.534534 -79.687728   214.290115  1.421849   
7677757  2022-12-23 22:00:00  37.534534 -79.687728   215.037491  1.185882   
7678961  2022-12-23 22:00:00  37.534534 -79.687728   214.830414  1.309650   
7681098  2022-12-23 23:00:00  37.534534 -79.687728   214.815125  1.310125   
7681465  2022-12-23 23:00:00  37.534534 -79.687728   214.

[5472 rows x 6 columns]

Longitude: -79.50472399558863
Latitude: 37.75702602378909
Grouped data:
                   Timestamp   Latitude  Longitude  Temperature  Humidity  \
232      2022-09-01 00:00:00  37.757026 -79.504724   208.350800  3.786933   
2573     2022-09-01 00:00:00  37.757026 -79.504724   208.307175  3.794195   
3528     2022-09-01 01:00:00  37.757026 -79.504724   208.457001  3.680294   
4919     2022-09-01 01:00:00  37.757026 -79.504724   208.244629  3.793959   
5847     2022-09-01 02:00:00  37.757026 -79.504724   208.596100  3.558174   
...                      ...        ...        ...          ...       ...   
7676842  2022-12-23 21:00:00  37.757026 -79.504724   214.415115  1.296849   
7679156  2022-12-23 22:00:00  37.757026 -79.504724   214.705414  1.309650   
7679644  2022-12-23 22:00:00  37.757026 -79.504724   214.974991  1.185882   
7680117  2022-12-23 23:00:00  37.757026 -79.504724   214.690125  1.310125   
7682457  2022-12-23 23:00:00  37.757026 -79.504724   213

[5472 rows x 6 columns]

Longitude: -79.36822545750755
Latitude: 35.58467047323055
Grouped data:
                   Timestamp  Latitude  Longitude  Temperature  Humidity  \
1017     2022-09-01 00:00:00  35.58467 -79.368225   205.413300  6.786933   
1797     2022-09-01 00:00:00  35.58467 -79.368225   205.682175  6.419195   
3192     2022-09-01 01:00:00  35.58467 -79.368225   205.557129  6.668959   
4265     2022-09-01 01:00:00  35.58467 -79.368225   205.519501  6.680294   
6634     2022-09-01 02:00:00  35.58467 -79.368225   205.658600  6.558174   
...                      ...       ...        ...          ...       ...   
7676055  2022-12-23 21:00:00  35.58467 -79.368225   208.665115  2.921849   
7677311  2022-12-23 22:00:00  35.58467 -79.368225   208.787491  2.935882   
7678448  2022-12-23 22:00:00  35.58467 -79.368225   209.142914  2.934650   
7680892  2022-12-23 23:00:00  35.58467 -79.368225   209.127625  2.935125   
7681669  2022-12-23 23:00:00  35.58467 -79.368225   209.006927  2.8

...

                   Timestamp   Latitude  Longitude  Temperature  Humidity  \
713      2022-09-01 00:00:00  35.555935 -75.663574   205.913300  6.286933   
2092     2022-09-01 00:00:00  35.555935 -75.663574   205.807175  6.544195   
3495     2022-09-01 01:00:00  35.555935 -75.663574   205.807129  6.418959   
4136     2022-09-01 01:00:00  35.555935 -75.663574   205.519501  6.930294   
6330     2022-09-01 02:00:00  35.555935 -75.663574   205.596100  6.683174   
...                      ...        ...        ...          ...       ...   
7676360  2022-12-23 21:00:00  35.555935 -75.663574   207.790115  3.546849   
7677973  2022-12-23 22:00:00  35.555935 -75.663574   207.724991  3.435882   
7678551  2022-12-23 22:00:00  35.555935 -75.663574   207.517914  3.434650   
7680588  2022-12-23 23:00:00  35.555935 -75.663574   207.502625  3.435125   
7681983  2022-12-23 23:00:00  35.555935 -75.663574   208.319427  3.140851   

             Pressure  
713      20299.997711  
2092     20599.997711  
349

[5472 rows x 6 columns]

Longitude: -75.47353575932908
Latitude: 36.62004252864905
Grouped data:
                   Timestamp   Latitude  Longitude  Temperature  Humidity  \
1109     2022-09-01 00:00:00  36.620043 -75.473536   207.038300  5.161933   
1695     2022-09-01 00:00:00  36.620043 -75.473536   206.682175  5.544195   
3098     2022-09-01 01:00:00  36.620043 -75.473536   206.807129  5.418959   
3682     2022-09-01 01:00:00  36.620043 -75.473536   206.582001  5.680294   
6725     2022-09-01 02:00:00  36.620043 -75.473536   206.783600  5.558174   
...                      ...        ...        ...          ...       ...   
7675965  2022-12-23 21:00:00  36.620043 -75.473536   208.352615  3.171849   
7677600  2022-12-23 22:00:00  36.620043 -75.473536   208.849991  2.810882   
7679001  2022-12-23 22:00:00  36.620043 -75.473536   209.330414  2.684650   
7680984  2022-12-23 23:00:00  36.620043 -75.473536   209.252625  2.685125   
7681588  2022-12-23 23:00:00  36.620043 -75.473536   209

[5472 rows x 6 columns]

Longitude: -75.31203431383983
Latitude: 35.03018831530443
Grouped data:
                   Timestamp   Latitude  Longitude  Temperature  Humidity  \
1023     2022-09-01 00:00:00  35.030188 -75.312034   205.538300  6.536933   
1782     2022-09-01 00:00:00  35.030188 -75.312034   204.932175  7.419195   
3186     2022-09-01 01:00:00  35.030188 -75.312034   204.932129  7.543959   
4295     2022-09-01 01:00:00  35.030188 -75.312034   205.144501  7.305294   
6639     2022-09-01 02:00:00  35.030188 -75.312034   205.158600  7.308174   
...                      ...        ...        ...          ...       ...   
7676058  2022-12-23 21:00:00  35.030188 -75.312034   207.352615  3.796849   
7677317  2022-12-23 22:00:00  35.030188 -75.312034   207.599991  3.685882   
7678388  2022-12-23 22:00:00  35.030188 -75.312034   207.267914  3.684650   
7680907  2022-12-23 23:00:00  35.030188 -75.312034   207.252625  3.685125   
7681663  2022-12-23 23:00:00  35.030188 -75.312034   207

[5472 rows x 6 columns]

Longitude: -75.14842613409076
Latitude: 38.38340257919188
Grouped data:
                   Timestamp   Latitude  Longitude  Temperature  Humidity  \
120      2022-09-01 00:00:00  38.383403 -75.148426   208.913300  4.036933   
2684     2022-09-01 00:00:00  38.383403 -75.148426   208.744675  4.044195   
3859     2022-09-01 01:00:00  38.383403 -75.148426   208.832001  3.805294   
4589     2022-09-01 01:00:00  38.383403 -75.148426   208.807129  3.918959   
5736     2022-09-01 02:00:00  38.383403 -75.148426   209.158600  3.558174   
...                      ...        ...        ...          ...       ...   
7676954  2022-12-23 21:00:00  38.383403 -75.148426   207.915115  3.171849   
7678824  2022-12-23 22:00:00  38.383403 -75.148426   210.267914  2.309650   
7679755  2022-12-23 22:00:00  38.383403 -75.148426   209.224991  2.685882   
7679996  2022-12-23 23:00:00  38.383403 -75.148426   210.252625  2.435125   
7682576  2022-12-23 23:00:00  38.383403 -75.148426   210

[5472 rows x 6 columns]



In [7]:
#selected area of coordinates 

long = -77.28391389676462
lat= 38.37780115022908

In [10]:
# Checking if there are datapoint at this alt and long coordinates 
selected_data = atmosphericData[(atmosphericData['Latitude'] == lat) & (atmosphericData['Longitude'] == long)]
if len(selected_data) > 0:
    print("There are rows with the specified latitude and longitude.")
else:
    print("There are no rows with the specified latitude and longitude.")

There are rows with the specified latitude and longitude.


## Converting Pressure and Calculating Altitude Levels 

The following resources are used to calculate altitude and used as Pressure Conversion   

* https://www.weather.gov/media/epz/wxcalc/pressureAltitude.pdf
* https://www.weather.gov/media/epz/wxcalc/pressureConversion.pdf
* https://www.weather.gov/media/epz/wxcalc/vaporPressure.pdf
* https://www.weather.gov/media/epz/wxcalc/tempConvert.pdf



In [15]:
def calculate_altitude(Pressure):
    altitude = (1 - ((Pressure/100) / 1013.25) ** 0.190284) * 145366.45  #  divide presssure 100 to convert it from Pascal to hpa
    return altitude
selected_data.loc[:, 'Altitude(feet)'] = calculate_altitude(selected_data['Pressure'])
selected_data

,Timestamp,Latitude,Longitude,Temperature,Humidity,Pressure,Altitude(feet)
26,2022-09-01 00:00:00,38.377801,-77.283914,208.663300,3.786933,16599.997711,42333.664589
2776,2022-09-01 00:00:00,38.377801,-77.283914,208.807175,3.669195,17199.997711,41635.181507
3777,2022-09-01 01:00:00,38.377801,-77.283914,209.019501,3.680294,17299.997711,41520.692525
5001,2022-09-01 01:00:00,38.377801,-77.283914,208.682129,3.793959,16899.997711,41981.912966
5641,2022-09-01 02:00:00,38.377801,-77.283914,209.221100,3.558174,17299.997711,41520.692525
...,...,...,...,...,...,...,...
7676982,2022-12-23 21:00:00,38.377801,-77.283914,213.102615,1.671849,28899.997711,30869.509134
7678906,2022-12-23 22:00:00,38.377801,-77.283914,214.580414,1.309650,30099.997711,29979.698829
7679857,2022-12-23 22:00:00,38.377801,-77.283914,213.537491,1.560882,35099.997711,26555.732465
7679900,2022-12-23 23:00:00,38.377801,-77.283914,214.502625,1.310125,30599.997711,29617.405535


### Merging Atmospheric Data with Ground weather data

In [18]:
Atm_data = selected_data.groupby('Timestamp').max().reset_index().rename(columns={'Timestamp': 'Datetime'})

#strip the last date from the atmospheric data to keep it consitent with ground surface weather data for the purpose of merging
Atm_data = Atm_data[~Atm_data['Datetime'].str.contains('2022-12-23')]
print ("RAP Atmospheric Data")
print(Atm_data.shape)

print ("Ground Surface Data")
grd_data = groundWeatehrData.groupby('Datetime').mean('Temperature')
print(grd_data.shape)



RAP Atmospheric Data
(2712, 7)
Ground Surface Data
(2674, 15)


In [19]:
combined_data = pd.merge(Atm_data, grd_data, on="Datetime")
#Renaming Ground Temprature and atmoshpheric temprature 
combined_data.rename(columns={'Temperature_x': 'atm_TMP', 'Temperature_y': 'grd_TMP'}, inplace=True)
combined_data.tail(2)

,Datetime,Latitude,Longitude,atm_TMP,Humidity,Pressure,Altitude(feet),grd_TMP,precp,humidity,...,dewPt,Image Present,Low/Mid Clouds (%),High Cirrus (%),Long-lived Contrails (Count),Contrail-Cirrus (Count),Day Low/Mid,Day Count Cirrs,Day Count Cont LL,Day Count Cont-Cirrus
2672,2022-12-21 23:00:00,38.377801,-77.283914,209.927338,2.825494,22299.997711,36944.977800,37.0,0.0,54.0,...,22.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2673,2022-12-22 00:00:00,38.377801,-77.283914,209.120422,2.950183,21699.997711,37815.374089,32.0,0.0,66.0,...,22.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Relative Humidity Calculation

The following resources are used to calculate RH and Saturated VP and Actual Vp

* https://www.weather.gov/media/epz/wxcalc/tempConvert.pdf  <<- Temp Convertion 
* https://www.weather.gov/media/epz/wxcalc/vaporPressure.pdf <<- RH AND pressure Calculation


RH = (e / es) * 100

Where:

- RH is the relative humidity expressed as a percentage
- e is the actual water vapor pressure
- es is the saturation vapor pressure at the same temperature.

The saturation vapor pressure (es) at a given temperature can be canculated using this formuala :

es = 6.11 * 10^((7.5 * T) / (237.3 + T))



In [1]:
#Converting TMP to degree of Celsius #
# combined_data['TMP_C'] = (combined_data['atm_TMP'] - 32) * 5/9
# combined_data['dewPT_C'] = (combined_data['dewPt'] - 32) * 5/9

In [25]:
# Calculate Saturated Vapor Pressure (Saturated_vp )
combined_data['Saturated_vp'] = 6.11 * 10**((7.5 * combined_data['atm_TMP']) / (237.3 + combined_data['atm_TMP']))


combined_data['RH_water'] = ((combined_data['Pressure'])/(combined_data['Saturated_vp'])) *100
combined_data.head(2)

,Datetime,Latitude,Longitude,atm_TMP,Humidity,Pressure,Altitude(feet),grd_TMP,precp,humidity,...,High Cirrus (%),Long-lived Contrails (Count),Contrail-Cirrus (Count),Day Low/Mid,Day Count Cirrs,Day Count Cont LL,Day Count Cont-Cirrus,Saturated_vp,RH_water,converted_vp
0,2022-09-01 05:00:00,38.377801,-77.283914,209.101562,3.561864,18599.997711,40513.701675,68.0,0.0,65.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,19914.068833,93.401293,185.999977
1,2022-09-01 06:00:00,38.377801,-77.283914,209.144577,3.317529,18599.997711,41293.312847,63.0,0.0,78.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,19931.690568,93.318716,185.999977


In [26]:
combined_data['Saturated_vp'] = 6.11 * 10**((7.5 * combined_data['atm_TMP']) / (237.3 + combined_data['atm_TMP']))
combined_data['converted_vp'] = combined_data['Pressure']/100

combined_data['RH_water2'] = ((combined_data['converted_vp'])/(combined_data['Saturated_vp']))*100 
combined_data.head(2)

,Datetime,Latitude,Longitude,atm_TMP,Humidity,Pressure,Altitude(feet),grd_TMP,precp,humidity,...,Long-lived Contrails (Count),Contrail-Cirrus (Count),Day Low/Mid,Day Count Cirrs,Day Count Cont LL,Day Count Cont-Cirrus,Saturated_vp,RH_water,converted_vp,RH_water2
0,2022-09-01 05:00:00,38.377801,-77.283914,209.101562,3.561864,18599.997711,40513.701675,68.0,0.0,65.0,...,NaN,NaN,NaN,NaN,NaN,NaN,19914.068833,93.401293,185.999977,0.934013
1,2022-09-01 06:00:00,38.377801,-77.283914,209.144577,3.317529,18599.997711,41293.312847,63.0,0.0,78.0,...,NaN,NaN,NaN,NaN,NaN,NaN,19931.690568,93.318716,185.999977,0.933187


## Validate

In [30]:
import numpy as np

# Constants
a_water = 17.2693882  # Replace with the actual value for a_water
a_ice = 21.8745484  # Replace with the actual value for a_ice
b_water = 35.86   # Replace with the actual value for b_water
b_ice = 7.66 
RH_ice = combined_data['RH_water']
TMP = combined_data['atm_TMP']# Replace with the actual value for b_ice

# Define a function to calculate the relative humidity with respect to water
def rh_water(a_water, a_ice, b_water, b_ice, TMP, RH_ice):
    return RH_ice / np.exp(a_water * TMP / (TMP - b_water)) - a_ice * (TMP / (TMP - b_ice))

# Calculate the relative humidity with respect to water
combined_data['Rh_water'] = rh_water(a_water, a_ice, b_water, b_ice, TMP, RH_ice)

# Display the updated
combined_data.head()

,Datetime,Latitude,Longitude,atm_TMP,Humidity,Pressure,Altitude(feet),grd_TMP,precp,humidity,...,Contrail-Cirrus (Count),Day Low/Mid,Day Count Cirrs,Day Count Cont LL,Day Count Cont-Cirrus,TMP_C,dewPT_C,Saturated_vp,RH_water,Rh_water
0,2022-09-01 05:00:00,38.377801,-77.283914,209.101562,3.561864,18599.997711,40513.701675,68.0,0.0,65.0,...,NaN,NaN,NaN,NaN,NaN,98.389757,13.333333,19914.068833,93.401293,-22.706348
1,2022-09-01 06:00:00,38.377801,-77.283914,209.144577,3.317529,18599.997711,41293.312847,63.0,0.0,78.0,...,NaN,NaN,NaN,NaN,NaN,98.413654,13.333333,19931.690568,93.318716,-22.706170
2,2022-09-01 07:00:00,38.377801,-77.283914,209.605942,3.188781,17399.997711,41635.179756,61.0,0.0,83.0,...,0.0,NaN,NaN,NaN,NaN,98.669968,13.333333,20121.465530,86.474803,-22.704271
3,2022-09-01 08:00:00,38.377801,-77.283914,209.579147,3.057698,34399.997711,41750.208978,60.0,0.0,86.0,...,0.0,NaN,NaN,NaN,NaN,98.655082,13.333333,20110.405577,171.055713,-22.704381
4,2022-09-01 09:00:00,38.377801,-77.283914,210.057922,3.078123,40799.997711,23391.045932,59.0,0.0,87.0,...,0.0,NaN,NaN,NaN,NaN,98.921068,12.777778,20308.747575,200.898640,-22.702418
